In [1]:
import findspark
findspark.init

<function findspark.init(spark_home=None, python_path=None, edit_rc=False, edit_profile=False)>

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("HousePriceModel").getOrCreate() 

23/01/29 16:45:30 WARN Utils: Your hostname, subbu resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/01/29 16:45:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/29 16:45:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy.stats import norm
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, cross_val_predict

In [4]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext

from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col

from pyspark.ml.regression import LinearRegression
from pyspark.mllib.evaluation import RegressionMetrics

from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, CrossValidatorModel
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.evaluation import RegressionEvaluator

In [5]:
from scipy import stats
from scipy.stats import skew 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [6]:
train = (spark.read.format("csv").option("header",True).option("inferSchema",True).load("train.csv"))

In [7]:
test = (spark.read.format("csv").option("header",True).option("inferSchema",True).load("test.csv"))

In [8]:
print(train.count())

1460


In [9]:
print(len(train.columns))

81


In [10]:
print(test.count())

1459


In [11]:
print(len(test.columns))

80


In [13]:
df=train.unionByName(test,allowMissingColumns=True)

In [14]:
df.limit(10).toPandas()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65,8450,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NA,Attchd,2003,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NA,NA,NA,0,2,2008,WD,Normal,208500
1,2,20,RL,80,9600,Pave,NA,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NA,NA,NA,0,5,2007,WD,Normal,181500
2,3,60,RL,68,11250,Pave,NA,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NA,NA,NA,0,9,2008,WD,Normal,223500
3,4,70,RL,60,9550,Pave,NA,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NA,NA,NA,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84,14260,Pave,NA,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NA,NA,NA,0,12,2008,WD,Normal,250000
5,6,50,RL,85,14115,Pave,NA,IR1,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1.5Fin,5,5,1993,1995,Gable,CompShg,VinylSd,VinylSd,None,0,TA,TA,Wood,Gd,TA,No,GLQ,732,Unf,0,64,796,GasA,Ex,Y,SBrkr,796,566,0,1362,1,0,1,1,1,1,TA,5,Typ,0,NA,Attchd,1993,Unf,2,480,TA,TA,Y,40,30,0,320,0,0,NA,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,RL,75,10084,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,1Fam,1Story,8,5,2004,2005,Gable,CompShg,VinylSd,VinylSd,Stone,186,Gd,TA,PConc,Ex,TA,Av,GLQ,1369,Unf,0,317,1686,GasA,Ex,Y,SBrkr,1694,0,0,1694,1,0,2,0,3,1,Gd,7,Typ,1,Gd,Attchd,2004,RFn,2,636,TA,TA,Y,255,57,0,0,0,0,NA,NA,NA,0,8,2007,WD,Normal,307000
7,8,60,RL,NA,10382,Pave,NA,IR1,Lvl,AllPub,Corner,Gtl,NWAmes,PosN,Norm,1Fam,2Story,7,6,1973,1973,Gable,CompShg,HdBoard,HdBoard,Stone,240,TA,TA,CBlock,Gd,TA,Mn,ALQ,859,BLQ,32,216,1107,GasA,Ex,Y,SBrkr,1107,983,0,2090,1,0,2,1,3,1,TA,7,Typ,2,TA,Attchd,1973,RFn,2,484,TA,TA,Y,235,204,228,0,0,0,NA,NA,Shed,350,11,2009,WD,Normal,200000
8,9,50,RM,51,6120,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Artery,Norm,1Fam,1.5Fin,7,5,1931,1950,Gable,CompShg,BrkFace,Wd Shng,None,0,TA,TA,BrkTil,TA,TA,No,Unf,0,Unf,0,952,952,GasA,Gd,Y,FuseF,1022,752,0,1774,0,0,2,0,2,2,TA,8,Min1,2,TA,Detchd,1931,Unf,2,468,Fa,TA,Y,90,0,205,0,0,0,NA,NA,NA,0,4,2008,WD,Abnorml,129900
9,10,190,RL,50,7420,Pave,NA,Reg,Lvl,AllPub,Corner,Gtl,BrkSide,Artery,Artery,2fmCon,1.5Unf,5,6,1939,1950,Gable,CompShg,MetalSd,MetalSd,None,0,TA,TA,BrkTil,TA,TA,No,GLQ,851,Unf,0,140,991,GasA,Ex,Y,SBrkr,1077,0,0,1077,1,0,1,0,2,2,TA,5,Typ,2,TA,Attchd,1939,RFn,1,205,Gd,TA,Y,0,4,0,0,0,

In [15]:
print(df.count())

2919


In [16]:
print(len(df.columns))

81


In [17]:
df.select('LotFrontage').agg(F.avg('LotFrontage')).show()

+-----------------+
| avg(LotFrontage)|
+-----------------+
|69.30579531442663|
+-----------------+



In [18]:
df.select('MasVnrArea').agg(F.avg('MasVnrArea')).show()

+------------------+
|   avg(MasVnrArea)|
+------------------+
|102.20131215469613|
+------------------+



In [19]:
df.select('GarageYrBlt').agg(F.avg('GarageYrBlt')).show()

+------------------+
|  avg(GarageYrBlt)|
+------------------+
|1978.1134057971014|
+------------------+



In [20]:
df.select('BsmtFinSF1').agg(F.avg('BsmtFinSF1')).show()

+-----------------+
|  avg(BsmtFinSF1)|
+-----------------+
|441.4232350925291|
+-----------------+



In [21]:
df.select('BsmtFinSF2').agg(F.avg('BsmtFinSF2')).show()

+-----------------+
|  avg(BsmtFinSF2)|
+-----------------+
|49.58224811514736|
+-----------------+



In [22]:
df.select('BsmtUnfSF').agg(F.avg('BsmtUnfSF')).show()

+-----------------+
|   avg(BsmtUnfSF)|
+-----------------+
|560.7721041809458|
+-----------------+



In [23]:
df.select('TotalBsmtSF').agg(F.avg('TotalBsmtSF')).show()

+------------------+
|  avg(TotalBsmtSF)|
+------------------+
|1051.7775873886224|
+------------------+



In [24]:
df.select('BsmtFullBath').agg(F.avg('BsmtFullBath')).show()

+-----------------+
|avg(BsmtFullBath)|
+-----------------+
|0.429893726431265|
+-----------------+



In [25]:
df.select('BsmtHalfBath').agg(F.avg('BsmtHalfBath')).show()

+-------------------+
|  avg(BsmtHalfBath)|
+-------------------+
|0.06136441549537196|
+-------------------+



In [26]:
df.select('GarageCars').agg(F.avg('GarageCars')).show()

+------------------+
|   avg(GarageCars)|
+------------------+
|1.7666209732693625|
+------------------+



In [27]:
df.select('GarageArea').agg(F.avg('GarageArea')).show()

+-----------------+
|  avg(GarageArea)|
+-----------------+
|472.8745716244003|
+-----------------+



In [29]:
df = df.withColumn("LotFrontage", F.when(F.col("LotFrontage") == 'NA',70).otherwise(F.col("LotFrontage"))) \
.withColumn("LotFrontage", F.col("LotFrontage").cast("int")) \
.withColumn("MasVnrArea", F.when(F.col("MasVnrArea") == 'NA',104).otherwise(F.col("MasVnrArea"))) \
.withColumn("MasVnrArea", F.col("MasVnrArea").cast("int")) \
.withColumn("GarageYrBlt", F.when(F.col("GarageYrBlt") == 'NA',1979).otherwise(F.col("GarageYrBlt"))) \
.withColumn("GarageYrBlt", F.col("GarageYrBlt").cast("int")) \
.withColumn("BsmtFinSF1", F.when(F.col("BsmtFinSF1") == 'NA',441).otherwise(F.col("BsmtFinSF1"))) \
.withColumn("BsmtFinSF1", F.col("BsmtFinSF1").cast("int")) \
.withColumn("BsmtFinSF2", F.when(F.col("BsmtFinSF2") == 'NA',50).otherwise(F.col("BsmtFinSF2"))) \
.withColumn("BsmtFinSF2", F.col("BsmtFinSF2").cast("int")) \
.withColumn("BsmtUnfSF", F.when(F.col("BsmtUnfSF") == 'NA',561).otherwise(F.col("BsmtUnfSF"))) \
.withColumn("BsmtUnfSF", F.col("BsmtUnfSF").cast("int")) \
.withColumn("TotalBsmtSF", F.when(F.col("TotalBsmtSF") == 'NA',1052).otherwise(F.col("TotalBsmtSF"))) \
.withColumn("TotalBsmtSF", F.col("TotalBsmtSF").cast("int")) \
.withColumn("BsmtFullBath", F.when(F.col("BsmtFullBath") == 'NA',0.43).otherwise(F.col("BsmtFullBath"))) \
.withColumn("BsmtFullBath", F.col("BsmtFullBath").cast("float")) \
.withColumn("BsmtHalfBath", F.when(F.col("BsmtHalfBath") == 'NA',0.06).otherwise(F.col("BsmtHalfBath"))) \
.withColumn("BsmtHalfBath", F.col("BsmtHalfBath").cast("float")) \
.withColumn("GarageCars", F.when(F.col("GarageCars") == 'NA',2).otherwise(F.col("GarageCars"))) \
.withColumn("GarageCars", F.col("GarageCars").cast("int")) \
.withColumn("GarageArea", F.when(F.col("GarageArea") == 'NA',473).otherwise(F.col("GarageArea"))) \
.withColumn("GarageArea", F.col("GarageArea").cast("int"))

In [30]:
def nullcount(df, col_name):
    nc = df.select(col_name).filter( 
        (F.col(col_name) == "NA") |
        (F.col(col_name) == "") |
        (F.col(col_name).isNull())
                                 ).count()
    return nc

In [31]:
dfcount = df.count()

In [32]:
for col_name in df.dtypes:
    nc = nullcount(df, col_name[0])

    if nc > 0:
        print("{} has {} % {}".format(col_name[0], nc, (nc/dfcount)*100))

MSZoning has 4 % 0.1370332305584104
Alley has 2721 % 93.21685508735868
Utilities has 2 % 0.0685166152792052
Exterior1st has 1 % 0.0342583076396026
Exterior2nd has 1 % 0.0342583076396026
MasVnrType has 24 % 0.8221993833504625
BsmtQual has 81 % 2.774922918807811
BsmtCond has 82 % 2.8091812264474134
BsmtExposure has 82 % 2.8091812264474134
BsmtFinType1 has 79 % 2.7064063035286057
BsmtFinType2 has 80 % 2.7406646111682083
Electrical has 1 % 0.0342583076396026
KitchenQual has 1 % 0.0342583076396026
Functional has 2 % 0.0685166152792052
FireplaceQu has 1420 % 48.646796848235695
GarageType has 157 % 5.378554299417608
GarageFinish has 159 % 5.4470709146968135
GarageQual has 159 % 5.4470709146968135
GarageCond has 159 % 5.4470709146968135
PoolQC has 2909 % 99.65741692360398
Fence has 2348 % 80.4385063377869
MiscFeature has 2814 % 96.40287769784173
SaleType has 1 % 0.0342583076396026
SalePrice has 1459 % 49.9828708461802


In [33]:
catcols = []
numcols = []
targetcol = ['SalePrice']
droppedcols = ['Id', 'Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature', 'Utilities', 'Condition2', 'RoofMatl', 'Heating']

In [34]:
for col_name in df.dtypes:
    if (col_name[0] not in targetcol + droppedcols):
        if col_name[1] == 'string':
            catcols.append(col_name[0])
        else:
            numcols.append(col_name[0])

In [35]:
if(len(df.columns) == len(targetcol) + len(droppedcols) + len(numcols) + len(catcols)):
    print("Number of variables are equal")
else: print("Number of variables are not equal")

Number of variables are equal


In [36]:
for cat_col in catcols:
    print(cat_col)
    df.groupBy(cat_col).count().orderBy(F.desc("count")).show()

MSZoning
+--------+-----+
|MSZoning|count|
+--------+-----+
|      RL| 2265|
|      RM|  460|
|      FV|  139|
|      RH|   26|
| C (all)|   25|
|      NA|    4|
+--------+-----+

Street
+------+-----+
|Street|count|
+------+-----+
|  Pave| 2907|
|  Grvl|   12|
+------+-----+

LotShape
+--------+-----+
|LotShape|count|
+--------+-----+
|     Reg| 1859|
|     IR1|  968|
|     IR2|   76|
|     IR3|   16|
+--------+-----+

LandContour
+-----------+-----+
|LandContour|count|
+-----------+-----+
|        Lvl| 2622|
|        HLS|  120|
|        Bnk|  117|
|        Low|   60|
+-----------+-----+

LotConfig
+---------+-----+
|LotConfig|count|
+---------+-----+
|   Inside| 2133|
|   Corner|  511|
|  CulDSac|  176|
|      FR2|   85|
|      FR3|   14|
+---------+-----+

LandSlope
+---------+-----+
|LandSlope|count|
+---------+-----+
|      Gtl| 2778|
|      Mod|  125|
|      Sev|   16|
+---------+-----+

Neighborhood
+------------+-----+
|Neighborhood|count|
+------------+-----+
|       NAmes|  4

In [37]:
binarycols = ['Street', 'CentralAir']

In [38]:
from pyspark.ml.feature import StringIndexer

In [41]:
print(catcols)

['MSZoning', 'Street', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'BldgType', 'HouseStyle', 'RoofStyle', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'SaleType', 'SaleCondition']


In [42]:
my_dict = {}

stringindexerobjs = []

stringindexeroutputnames = []

oheinputnames = []

oheoutputnames = []

for col_name in catcols:
    my_dict[col_name+"_index_obj"] = StringIndexer() \
    .setHandleInvalid("skip") \
    .setInputCol(col_name) \
    .setOutputCol(col_name+"_indexed")
    
    stringindexerobjs.append(my_dict.get(col_name+"_index_obj"))
    stringindexeroutputnames.append(col_name+"_indexed")

    
    if col_name not in binarycols:
        oheinputnames.append(col_name+"_indexed")
        oheoutputnames.append(col_name+"_ohe")

In [43]:
print(stringindexerobjs)
print(len(stringindexerobjs))

[StringIndexer_4ad391586c80, StringIndexer_9122bd4f8c16, StringIndexer_790c77a21509, StringIndexer_a4efce8b24be, StringIndexer_96e1abf45122, StringIndexer_75a5ccfd8878, StringIndexer_cf9b39eb8081, StringIndexer_5de34598045d, StringIndexer_6afaa74239f4, StringIndexer_b668fa57c827, StringIndexer_fb49941825df, StringIndexer_ff94637124a5, StringIndexer_51825299bbc0, StringIndexer_97500921b719, StringIndexer_7a9f27675732, StringIndexer_8dd00d698d09, StringIndexer_a87d0b2e9962, StringIndexer_651dd0dec108, StringIndexer_c464c76c8ec6, StringIndexer_073437716c4a, StringIndexer_b8a402600a38, StringIndexer_1665ed302211, StringIndexer_7cfbeee37af4, StringIndexer_6fd715c268e4, StringIndexer_414ebd6dbd16, StringIndexer_84164d1bfcf0, StringIndexer_bd6ba4e14a57, StringIndexer_1de663789828, StringIndexer_6a89b85f5df4, StringIndexer_0d023f443955, StringIndexer_d9a852326fca, StringIndexer_25d8160c2435, StringIndexer_5b0af0fcd303, StringIndexer_2b090129fb15]
34


In [44]:
print(stringindexeroutputnames)
print(len(stringindexeroutputnames))

['MSZoning_indexed', 'Street_indexed', 'LotShape_indexed', 'LandContour_indexed', 'LotConfig_indexed', 'LandSlope_indexed', 'Neighborhood_indexed', 'Condition1_indexed', 'BldgType_indexed', 'HouseStyle_indexed', 'RoofStyle_indexed', 'Exterior1st_indexed', 'Exterior2nd_indexed', 'MasVnrType_indexed', 'ExterQual_indexed', 'ExterCond_indexed', 'Foundation_indexed', 'BsmtQual_indexed', 'BsmtCond_indexed', 'BsmtExposure_indexed', 'BsmtFinType1_indexed', 'BsmtFinType2_indexed', 'HeatingQC_indexed', 'CentralAir_indexed', 'Electrical_indexed', 'KitchenQual_indexed', 'Functional_indexed', 'GarageType_indexed', 'GarageFinish_indexed', 'GarageQual_indexed', 'GarageCond_indexed', 'PavedDrive_indexed', 'SaleType_indexed', 'SaleCondition_indexed']
34


In [45]:
print(oheinputnames)
print(len(oheinputnames))

['MSZoning_indexed', 'LotShape_indexed', 'LandContour_indexed', 'LotConfig_indexed', 'LandSlope_indexed', 'Neighborhood_indexed', 'Condition1_indexed', 'BldgType_indexed', 'HouseStyle_indexed', 'RoofStyle_indexed', 'Exterior1st_indexed', 'Exterior2nd_indexed', 'MasVnrType_indexed', 'ExterQual_indexed', 'ExterCond_indexed', 'Foundation_indexed', 'BsmtQual_indexed', 'BsmtCond_indexed', 'BsmtExposure_indexed', 'BsmtFinType1_indexed', 'BsmtFinType2_indexed', 'HeatingQC_indexed', 'Electrical_indexed', 'KitchenQual_indexed', 'Functional_indexed', 'GarageType_indexed', 'GarageFinish_indexed', 'GarageQual_indexed', 'GarageCond_indexed', 'PavedDrive_indexed', 'SaleType_indexed', 'SaleCondition_indexed']
32


In [46]:
print(oheoutputnames)
print(len(oheoutputnames))

['MSZoning_ohe', 'LotShape_ohe', 'LandContour_ohe', 'LotConfig_ohe', 'LandSlope_ohe', 'Neighborhood_ohe', 'Condition1_ohe', 'BldgType_ohe', 'HouseStyle_ohe', 'RoofStyle_ohe', 'Exterior1st_ohe', 'Exterior2nd_ohe', 'MasVnrType_ohe', 'ExterQual_ohe', 'ExterCond_ohe', 'Foundation_ohe', 'BsmtQual_ohe', 'BsmtCond_ohe', 'BsmtExposure_ohe', 'BsmtFinType1_ohe', 'BsmtFinType2_ohe', 'HeatingQC_ohe', 'Electrical_ohe', 'KitchenQual_ohe', 'Functional_ohe', 'GarageType_ohe', 'GarageFinish_ohe', 'GarageQual_ohe', 'GarageCond_ohe', 'PavedDrive_ohe', 'SaleType_ohe', 'SaleCondition_ohe']
32


In [47]:
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import VectorAssembler

In [48]:
encoder = OneHotEncoder() \
.setInputCols(oheinputnames) \
.setOutputCols(oheoutputnames)

In [49]:
nottohotcoded = list(set(stringindexeroutputnames).difference(set(oheinputnames)))
print(nottohotcoded)

['Street_indexed', 'CentralAir_indexed']


In [50]:
assembler = VectorAssembler() \
.setHandleInvalid("skip") \
.setInputCols(numcols+ nottohotcoded+ oheoutputnames) \
.setOutputCol("unscaled_features")

In [51]:
from pyspark.ml.feature import StandardScaler

In [52]:
scaler = StandardScaler() \
.setInputCol("unscaled_features") \
.setOutputCol("features")

In [53]:
submissiondf = df.where(col("SalePrice").isNull())

In [54]:
modeldf = df.where(col("SalePrice").isNotNull())

In [55]:
submissiondf.limit(5).toPandas()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1461,20,RH,80,11622,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0,TA,TA,CBlock,TA,TA,No,Rec,468,LwQ,144,270,882,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NA,Attchd,1961,Unf,1,730,TA,TA,Y,140,0,0,0,120,0,NA,MnPrv,NA,0,6,2010,WD,Normal,NaN
1,1462,20,RL,81,14267,Pave,NA,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108,TA,TA,CBlock,TA,TA,No,ALQ,923,Unf,0,406,1329,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NA,Attchd,1958,Unf,1,312,TA,TA,Y,393,36,0,0,0,0,NA,NA,Gar2,12500,6,2010,WD,Normal,NaN
2,1463,60,RL,74,13830,Pave,NA,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0,TA,TA,PConc,Gd,TA,No,GLQ,791,Unf,0,137,928,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997,Fin,2,482,TA,TA,Y,212,34,0,0,0,0,NA,MnPrv,NA,0,3,2010,WD,Normal,NaN
3,1464,60,RL,78,9978,Pave,NA,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20,TA,TA,PConc,TA,TA,No,GLQ,602,Unf,0,324,926,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998,Fin,2,470,TA,TA,Y,360,36,0,0,0,0,NA,NA,NA,0,6,2010,WD,Normal,NaN
4,1465,120,RL,43,5005,Pave,NA,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0,Gd,TA,PConc,Gd,TA,No,ALQ,263,Unf,0,1017,1280,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NA,Attchd,1992,RFn,2,506,TA,TA,Y,0,82,0,0,144,0,NA,NA,NA,0,1,2010,WD,Normal,NaN


In [56]:
modeldf.limit(5).toPandas()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65,8450,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1.0,0.0,2,1,3,1,Gd,8,Typ,0,NA,Attchd,2003,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NA,NA,NA,0,2,2008,WD,Normal,208500
1,2,20,RL,80,9600,Pave,NA,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0.0,1.0,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NA,NA,NA,0,5,2007,WD,Normal,181500
2,3,60,RL,68,11250,Pave,NA,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1.0,0.0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NA,NA,NA,0,9,2008,WD,Normal,223500
3,4,70,RL,60,9550,Pave,NA,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1.0,0.0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NA,NA,NA,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84,14260,Pave,NA,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1.0,0.0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NA,NA,NA,0,12,2008,WD,Normal,250000


In [58]:
from pyspark.ml.regression import GBTRegressor
estimator = GBTRegressor(labelCol = targetcol[0])

In [59]:
from pyspark.ml import Pipeline

In [60]:
pipeline_obj = Pipeline().setStages(stringindexerobjs + [encoder, assembler, scaler, estimator])

In [61]:
train_df, test_df = modeldf.randomSplit([.8, .2], seed =142)

In [62]:
train_df.count()

1163

In [63]:
test_df.count()

297

In [64]:
pipeline_model = pipeline_obj.fit(train_df)

In [65]:
predictiondf = pipeline_model.transform(test_df)

In [66]:
predictiondf.select("SalePrice", "prediction").show()

+---------+------------------+
|SalePrice|        prediction|
+---------+------------------+
|   140000|154894.01384119183|
|   129900|183806.65751608292|
|   129500| 143479.0007706752|
|   157000|150722.64905722594|
|   154000|153344.50285048567|
|   207500|184477.29990063622|
|   165500| 156147.7291032351|
|   160000|151963.02108115493|
|   239686| 339562.1712050445|
|   127000|116275.84877230343|
|   110000| 93446.56640190855|
|   385000|  794193.964871812|
|   101000| 83731.09300683196|
|   245000|235780.65149632152|
|   260000|284251.62514613394|
|   204750|195686.25333934475|
|   214000| 230843.1420089899|
|   198900|204316.91386524081|
|   169500|193609.23940570257|
|   180000|154609.62961301126|
+---------+------------------+
only showing top 20 rows



In [67]:
from pyspark.ml.evaluation import RegressionEvaluator

In [68]:
evaluator = RegressionEvaluator(labelCol=targetcol[0], metricName='r2')

In [69]:
evaluator.evaluate(predictiondf)

0.7153289714673818

In [70]:
transformeddf = pipeline_model.transform(submissiondf)

In [71]:
transformeddf.limit(5).toPandas()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,MSZoning_indexed,Street_indexed,LotShape_indexed,LandContour_indexed,LotConfig_indexed,LandSlope_indexed,Neighborhood_indexed,Condition1_indexed,BldgType_indexed,HouseStyle_indexed,RoofStyle_indexed,Exterior1st_indexed,Exterior2nd_indexed,MasVnrType_indexed,ExterQual_indexed,ExterCond_indexed,Foundation_indexed,BsmtQual_indexed,BsmtCond_indexed,BsmtExposure_indexed,BsmtFinType1_indexed,BsmtFinType2_indexed,HeatingQC_indexed,CentralAir_indexed,Electrical_indexed,KitchenQual_indexed,Functional_indexed,GarageType_indexed,GarageFinish_indexed,GarageQual_indexed,GarageCond_indexed,PavedDrive_indexed,SaleType_indexed,SaleCondition_indexed,MSZoning_ohe,LotShape_ohe,LandContour_ohe,LotConfig_ohe,LandSlope_ohe,Neighborhood_ohe,Condition1_ohe,BldgType_ohe,HouseStyle_ohe,RoofStyle_ohe,Exterior1st_ohe,Exterior2nd_ohe,MasVnrType_ohe,ExterQual_ohe,ExterCond_ohe,Foundation_ohe,BsmtQual_ohe,BsmtCond_ohe,BsmtExposure_ohe,BsmtFinType1_ohe,BsmtFinType2_ohe,HeatingQC_ohe,Electrical_ohe,KitchenQual_ohe,Functional_ohe,GarageType_ohe,GarageFinish_ohe,GarageQual_ohe,GarageCond_ohe,PavedDrive_ohe,SaleType_ohe,SaleCondition_ohe,unscaled_features,features,prediction
0,1461,20,RH,80,11622,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0,TA,TA,CBlock,TA,TA,No,Rec,468,LwQ,144,270,882,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NA,Attchd,1961,Unf,1,730,TA,TA,Y,140,0,0,0,120,0,NA,MnPrv,NA,0,6,2010,WD,Normal,NaN,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"(0.0, 0.0, 0.0, 1.0)","(1.0, 0.0, 0.0)","(1.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0)","(1.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 0.0, 1.0, 0.0)","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(20.0, 80.0, 11622.0, 5.0, 6.0, 1961.0, 1961.0...","(0.47351732827060145, 3.5423261350910664, 1.16...",118996.137597
1,1462,20,RL,81,14267,Pave,NA,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108,TA,TA,CBlock,TA,TA,No,ALQ,923,Unf,0,406,1329,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NA,Attchd,1958,Unf,1,312,TA,TA,Y,393,36,0,0,0,0,NA,NA,Gar2,12500,6,2010,WD,Normal,NaN,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"(1.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0)","(1.0, 0.0, 0.0

In [72]:
submission = transformeddf.select("Id", "prediction")

In [73]:
submission.limit(5).toPandas()

,Id,prediction
0,1461,118996.137597
1,1462,145808.210528
2,1463,170254.953205
3,1464,189777.160427
4,1465,217631.827804


In [76]:
pipeline_model.write().overwrite().save("/home/subbu/Documents/model")

In [79]:
# import PipelineModel from pyspark.ml package
from pyspark.ml import PipelineModel

# load the model from the location it is stored
# The loaded model acts as PipelineModel
pipemodel = PipelineModel.load("/home/subbu/Documents/model")

# use the PipelineModel object to perform prediciton on test data. 
# Use .transform() to perfrom prediction
prediction = pipemodel.transform(submissiondf)

# print the results
prediction.limit(5).toPandas()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,MSZoning_indexed,Street_indexed,LotShape_indexed,LandContour_indexed,LotConfig_indexed,LandSlope_indexed,Neighborhood_indexed,Condition1_indexed,BldgType_indexed,HouseStyle_indexed,RoofStyle_indexed,Exterior1st_indexed,Exterior2nd_indexed,MasVnrType_indexed,ExterQual_indexed,ExterCond_indexed,Foundation_indexed,BsmtQual_indexed,BsmtCond_indexed,BsmtExposure_indexed,BsmtFinType1_indexed,BsmtFinType2_indexed,HeatingQC_indexed,CentralAir_indexed,Electrical_indexed,KitchenQual_indexed,Functional_indexed,GarageType_indexed,GarageFinish_indexed,GarageQual_indexed,GarageCond_indexed,PavedDrive_indexed,SaleType_indexed,SaleCondition_indexed,MSZoning_ohe,LotShape_ohe,LandContour_ohe,LotConfig_ohe,LandSlope_ohe,Neighborhood_ohe,Condition1_ohe,BldgType_ohe,HouseStyle_ohe,RoofStyle_ohe,Exterior1st_ohe,Exterior2nd_ohe,MasVnrType_ohe,ExterQual_ohe,ExterCond_ohe,Foundation_ohe,BsmtQual_ohe,BsmtCond_ohe,BsmtExposure_ohe,BsmtFinType1_ohe,BsmtFinType2_ohe,HeatingQC_ohe,Electrical_ohe,KitchenQual_ohe,Functional_ohe,GarageType_ohe,GarageFinish_ohe,GarageQual_ohe,GarageCond_ohe,PavedDrive_ohe,SaleType_ohe,SaleCondition_ohe,unscaled_features,features,prediction
0,1461,20,RH,80,11622,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0,TA,TA,CBlock,TA,TA,No,Rec,468,LwQ,144,270,882,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NA,Attchd,1961,Unf,1,730,TA,TA,Y,140,0,0,0,120,0,NA,MnPrv,NA,0,6,2010,WD,Normal,NaN,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"(0.0, 0.0, 0.0, 1.0)","(1.0, 0.0, 0.0)","(1.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0)","(1.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 0.0, 1.0, 0.0)","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(20.0, 80.0, 11622.0, 5.0, 6.0, 1961.0, 1961.0...","(0.47351732827060145, 3.5423261350910664, 1.16...",118996.137597
1,1462,20,RL,81,14267,Pave,NA,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108,TA,TA,CBlock,TA,TA,No,ALQ,923,Unf,0,406,1329,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NA,Attchd,1958,Unf,1,312,TA,TA,Y,393,36,0,0,0,0,NA,NA,Gar2,12500,6,2010,WD,Normal,NaN,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"(1.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0)","(1.0, 0.0, 0.0